In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

api_keys={
    'google_api_key':os.getenv('google_api_key'),
    'pse':os.getenv('pse'),
    'openai_api_key':os.getenv('openai_api_key'),
    'composio_key':os.getenv('composio_api_key'),
    'notion_api_key':os.getenv('notion_integration_token')  # Add your Notion API key here
}

In [4]:
from composio_langgraph import Action, ComposioToolSet, App
from utils.composio_tools_agent import Composio_agent
from pydantic_ai import Agent, format_as_xml,RunContext
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.providers.google import GoogleProvider
from langchain_openai import ChatOpenAI
from datetime import datetime
from pydantic_ai.messages import ModelMessage
from pydantic import BaseModel,Field
from dataclasses import dataclass
import nest_asyncio
nest_asyncio.apply()

In [10]:
from pydantic_ai.mcp import MCPServerStreamableHTTP, MCPServerSSE, MCPServerStdio
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai import Agent
from pydantic_ai.messages import ModelMessage
from dataclasses import dataclass

@dataclass
class Message_state:
    messages:list[ModelMessage]

class NotionAgent_MCP:
    def __init__(self, mcp_server_url:str, api_keys:dict):
        self.mcp_server_url=mcp_server_url
        self.api_keys=api_keys
        self.tools=ComposioToolSet(api_key=api_keys['composio_key'])
        self.tool_shemas={
            'Notion Manager':{tool.name:tool for tool in self.tools.get_action_schemas(apps=[App.NOTION])}}
        self.memory=Message_state(messages=[])
        self.llm=OpenAIModel('gpt-4.1-mini',provider=OpenAIProvider(api_key=api_keys['openai_api_key']))
        self.mcp_server=MCPServerStreamableHTTP(self.mcp_server_url)
        self.agent=Agent(self.llm, mcp_servers=[self.mcp_server])
        self._mcp_context_manager = None
        self._is_connected = False

    async def connect(self):
        """Establish persistent connection to MCP server"""
        if not self._is_connected:
            self._mcp_context_manager = self.agent.run_mcp_servers()
            await self._mcp_context_manager.__aenter__()
            self._is_connected = True
            print("Connected to MCP server")

    async def disconnect(self):
        """Close the MCP server connection"""
        if self._is_connected and self._mcp_context_manager:
            await self._mcp_context_manager.__aexit__(None, None, None)
            self._is_connected = False
            self._mcp_context_manager = None
            print("Disconnected from MCP server")
    
    async def chat(self, query: str):
        """Execute query using persistent MCP connection"""
        if not self._is_connected:
            await self.connect()
        
        result = self.agent.run_sync(query, message_history=self.memory.messages)
        self.memory.messages = result.all_messages()
        return result.output

    def reset(self):
        """Reset conversation memory"""
        self.memory.messages = []

    async def __aenter__(self):
        """Async context manager entry"""
        await self.connect()
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Async context manager exit"""
        await self.disconnect()

In [11]:
agent2=NotionAgent_MCP(mcp_server_url=os.getenv('mcp_server_url'), api_keys=api_keys)

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Notion-agent\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions


  warnings.warn(


In [12]:
# Option 1: Manual connection management
await agent2.connect()  # Connect once
result1 = await agent2.chat("Hello, what can you help me with?")
print("Result 1:", result1)

# result2 = await agent2.main("Create a new page in Notion with title 'Test Page'")
# print("Result 2:", result2)

# When done, disconnect
# await agent2.disconnect()

Connected to MCP server
Result 1: Hello! I can assist you with various tasks related to Notion, such as:

- Creating, updating, and deleting pages and databases.
- Adding content blocks to pages.
- Fetching information about pages, databases, users, and comments.
- Querying databases and updating database rows.
- Managing comments and discussions within Notion.

If you have any specific requests or questions about using Notion, feel free to ask! How can I assist you today?


In [ ]:
await agent2.chat('hello')

'Hello! How can I assist you today?'

In [ ]:
# Option 2: Using async context manager (recommended)
# This automatically connects and disconnects
async with NotionAgent_MCP(mcp_server_url=os.getenv('mcp_server_url'), api_keys=api_keys) as agent3:
    result1 = await agent3.main("Hello, what can you help me with?")
    print("Result 1:", result1)
    
    result2 = await agent3.main("What's the current date?")
    print("Result 2:", result2)
    
    result3 = await agent3.main("List my recent Notion pages")
    print("Result 3:", result3)

# Connection is automatically closed when exiting the context


In [6]:
# from notion_agent import Notionagent


In [ ]:

# agent=Notionagent(mcp_server_url=os.getenv('mcp_server_url'),api_keys=api_keys)



c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Notion-agent\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions

Give Feedback / Get Help:
    On GitHub: https://github.com/ComposioHQ/composio/issues/new
    On Discord: https://dub.composio.dev/discord
    On Email: tech@composio.dev
    Talk to us on Intercom: https://composio.dev
    Book a call with us: https://composio.dev/redirect?url=https://calendly.com/composiohq/support?utm_source=py-sdk-logs&utm_campaign=calendly
If you need to debug this error, set `COMPOSIO_LOGGING_LEVEL=debug`.
  warnings.warn(


In [9]:
await agent.chat('hello')

'Hello! How can I assist you today with Notion?'